# Machine Learning Potentials and Molecular Dynamics Simulations for Materials Science

## Introduction: Empirical Potentials and Molecular Dynamics

Molecular dynamics (MD) simulations are a cornerstone of computational materials science, enabling us to model the time evolution of atoms based on classical mechanics. The trajectory of each atom is computed by solving Newton's equations of motion:

$$
F = m\, a,
$$
where the force is given by the gradient of the potential energy, $U$, with respect to the coordinate, $\vec r_i$ of each atom $i$.

Hence,
$$
-\nabla_{\mathbf{r}_i} U(\mathbf{r}_1, \dots, \mathbf{r}_N) = m_i \frac{d^2\mathbf{r}_i}{dt^2}
$$

### A Brief History of Potentials

- **1930s–1960s**: Early MD simulations used hard-sphere or Lennard-Jones (LJ) potentials.
- **1970s–1980s**: Development of many-body potentials like **Embedded Atom Method (EAM)** and **Tersoff**.
- **1990s–2010s**: Reactive force fields (ReaxFF) and tight-binding approaches added chemical reactivity.
- **2020s–now**: Machine learning potentials (MLPs) like **GAP**, **NequIP**, and **UF3** offer near-quantum mechanical accuracy.

### Why Machine Learning Potentials?

Machine-learned interatomic potentials interpolate the quantum mechanical potential energy surface (described by density functional theory or DFT) using high-dimensional, non-linear functions:
- The goal is for the accuracy of MLIPs to approach that of DFT while being as fast like empirical force fields.
- Applicable to large-scale MD for melting, diffusion, and phase transitions.

In this notebook, we train a UF3 model for tungsten and use it to simulate melting with LAMMPS.

In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Uncomment and run if not already installed

# Install UF3
#!pip install uf3@monk04-patch-3

# Install LAMMPS with UF3 support

In [ ]:
from uf3.data import io
from uf3.data import geometry
from uf3.data import composition
from uf3.representation import bspline
from uf3.representation import process
from uf3.regression import least_squares
from uf3.forcefield import calculator
from uf3.forcefield import lammps
from uf3.util import parallel
from uf3.util import plotting
from uf3.util import plotting3d
from uf3.util import plot_slices_3b

# $\text{UF}_{2,3}$ Demo: Elemental tungsten

**Data split**
- Training set: 1939 configurations (stratified 20% of the dataset)

- Holdout: 7754 configurations (remaining 80%)

**Inputs**
- ```w-14.xyz``` (30 mb)
- ```training_idx.txt``` (10 kb, included for reproducibility purposes)

**Outputs**
- ```df_features_uf23.h5``` (650 mb)
- ```model_uf23.json``` (3 kb)

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

 Step         | Estimated Time 
:-------------|:--------------
Preprocessing | 10 seconds
Featurization | 30 core-minutes (parallelizable)
Training      | 4 seconds
Prediction    | 3 seconds
Plotting      | 5 seconds

# User Parameters

```element_list (list)```: list of element symbols

```degree (int)```: truncation of many-body expansion. A value of 3 yields a two-and-three-body potential.

In [ ]:
element_list = ['W']
degree = 3

Initialize the ```ChemicalSystem``` and inspect interactions.

Elements involved in each interactions are sorted by electronegativity.

In [ ]:
chemical_system = composition.ChemicalSystem(element_list=element_list,
                                             degree=degree)
print(chemical_system)

In [ ]:
print("Trios:", chemical_system.interactions_map[3])

# Selecting cutoffs

```r_min_map (dict)```: map of minimum pair distance per interaction (angstroms). 
    If unspecified, defaults to 1.0 for all interactions.
    
```r_max_map (dict)```: map of maximum pair distance per interaction (angstroms). 
    If unspecified, defaults to 6.0 angstroms for all interactions, which probably encompasses at least 2nd-nearest neighbors.
    
```resolution_map (dict)```: map of resolution (number of knot intervals) per interaction. 
    For the cubic basis, the number of basis functions equals three more than the number of knot intervals.
    This is, in turn, negated by ```trailing_trim```.
    If unspecified, defaults to 20 for all two-body interactions and 5 for three-body interactions.
    
```trailing_trim (int)```: number of trailing basis functions to trim, defaults to 3.
 - ```= 0```: hard cutoff at ```r_max```
 - ```= 1```: function goes to zero at ```r_max```
 - ```= 2```: first derivative goes to zero at ```r_max```
 - ```= 3```: second derivative goes to zero at ```r_max```
 
```leading_trim (int)```: similar for leading basis functions (small distances), defaults to 0 for 2-body and 3 for 3-body

**Note: the demo's resolution and cutoffs (3.5-3.5-7.0Å, 6-6-12) are small to reduce runtime and filesize.**

**Results in the manuscript use (4.25-4.25-8.5Å, 10-10-20), requiring about 20 core-minutes and 6 gb.**

## Sensible values can be chosen based on the results from the two-body demo.
**r_min and r_max should span the region near the two-body potential minimum
and any inflection points.**

**High values for r_max increase the size of neighbor lists and slows down calculations.
Sensible values for r_max for the two-body terms can be quickly estimated
through grid-search in the two-body demo.**

**r_max for the three-body can be slightly lower based on the assumption that
higher-order terms are most important for nearest neighbors.**

In [ ]:
r_min_map = {("W", "W"): 0.001,
             ("W", "W", "W"): [1.0, 1.0, 1.0],
            }
r_max_map = {("W", "W"): 5.5,
             ("W", "W", "W"): [3.5, 3.5, 7.0],
            }
resolution_map = {("W", "W"): 15,
                  ("W", "W", "W"): [9, 9, 12],
                 }
#trailing_trim = {2: 3, 3: 3}  # 3 for 2-body, 3 for 3-body
#leading_trim = {2: 0, 3: 3}  # 0 for 2-body, 3 for 3-body

trailing_trim = 3  # 3 for 2-body, 3 for 3-body
leading_trim =  0  # 0 for 2-body, 3 for 3-body

# Integer values are also accepted for the trim parameters, in which case the
# same value is used for both 2-body and 3-body.

# Demo parameters
```n_cores```: number of workers to use in parallel for feature generation

```data_filename```: filename of reference data including geometries, energies, forces, ...

```filename```: filename to save features dataframe.

```table_template```: format string for table names in feature dataframe.

In [ ]:
n_cores = os.cpu_count()
print("Number of cores:", n_cores)

In [ ]:
example_directory = os.getcwd()
data_filename = os.path.join(example_directory, "w-14.xyz")

In [ ]:
filename = "df_features.h5"
table_template = "features_{}"

# Initialize basis

In [ ]:
bspline_config = bspline.BSplineBasis(chemical_system,
                                      r_min_map=r_min_map,
                                      r_max_map=r_max_map,
                                      resolution_map=resolution_map,
                                      leading_trim=leading_trim,
                                      trailing_trim=trailing_trim)

```bspline_config.get_interaction_partitions()``` yields the number of coefficients for each n-body interaction (one-body terms, two-body terms, three-body terms, ...) as well as the starting index in the coefficient vector for each interaction.

In [ ]:
bspline_config.get_interaction_partitions()[0]

In [ ]:
bspline_config.get_interaction_partitions()[1]

# Load data

In [ ]:
data_coordinator = io.DataCoordinator()
data_coordinator.dataframe_from_trajectory(data_filename,
                                           prefix='dft')
df_data = data_coordinator.consolidate()
print("Number of energies:", len(df_data))
print("Number of forces:", int(np.sum(df_data["size"]) * 3))

In [ ]:
df_data.head()

In [ ]:
# Remove all the data of size 1 as they copntain no forces and this results in an error in the current UF3 implementation
df_data = df_data[df_data["size"] > 1]
df_data = df_data.reset_index(drop=True)
print("Number of energies:", len(df_data))
print("Number of forces:", int(np.sum(df_data["size"]) * 3))
df_data.tail()

# Examine pair distance distribution
Useful step that serves as a sanity check for selected cutoffs and resolution.

In [ ]:
from uf3.data import analyze
from tqdm.auto import tqdm

In [ ]:
analyzer = analyze.DataAnalyzer(chemical_system, 
                                r_cut=10.0,
                                bins=0.01)

In [ ]:
atoms_key = data_coordinator.atoms_key
histogram_slice = np.random.choice(np.arange(len(df_data)),
                                   min(1000, len(df_data)),
                                   replace=False)
df_slice = df_data[atoms_key].iloc[histogram_slice]
analyzer.load_entries(df_slice)

In [ ]:
analysis = analyzer.analyze()

In [ ]:
canvases = plotting.plot_pair_distributions(analysis, show_cutoffs=False, figsize=(3.5, 1.5))
for fig, ax in canvases:
    fig.set_dpi(120)
    fig.tight_layout()
    ax.set_ylabel("")
    # fig.show()

# Compute energy and force features



In [ ]:
representation = process.BasisFeaturizer(bspline_config)

In [ ]:
client = ProcessPoolExecutor(max_workers=n_cores)

In [ ]:
representation.batched_to_hdf(filename,
                              df_data,
                              client,
                              n_jobs = n_cores,
                              batch_size=50,
                              progress="bar",
                              table_template=table_template)

# Fit model

### Split data into training and testing set

In [ ]:
# create training and test index using a random split

train_index = np.random.choice(df_data.index, size=int(len(df_data) * 0.8), replace=False)
test_index = np.array(list(set(df_data.index) - set(train_index)))
train_index = np.sort(train_index)
test_index = np.sort(test_index)
print("Number of training points:", len(train_index))
print("Number of test points:", len(test_index))
print("Number of training forces:", int(np.sum(df_data.loc[train_index]["size"]) * 3))
print("Number of test forces:", int(np.sum(df_data.loc[test_index]["size"]) * 3))

### Regularized linear regression

In [ ]:
regularizer = bspline_config.get_regularization_matrix(ridge_1b=0.0,
                                                       ridge_2b=0.0,
                                                       ridge_3b=1e-8,
                                                       curvature_2b=1e-8,
                                                       curvature_3b=0.0)

model = least_squares.WeightedLinearModel(bspline_config,
                                          regularizer=regularizer)

# Fit with energies and force

Train with the training index that includes 80% of the dataset

In [ ]:
model.fit_from_file(filename, 
                    df_data.index[train_index],
                    weight=0.8, 
                    batch_size=2500,
                    energy_key="energy", 
                    progress="bar")

In [ ]:
pair = ("W", "W")

r_target = analysis["lower_bounds"][pair]
model.fix_repulsion_2b(pair, 
                       r_target=r_target,
                       min_curvature=0.0)

solutions = least_squares.arrange_coefficients(model.coefficients,
                                               bspline_config)
coefficients = solutions[("W", "W")]
knot_sequence = bspline_config.knots_map[("W", "W")]
fig, ax = plotting.visualize_splines(coefficients, knot_sequence)
plt.vlines([0.0], -100, 100, color="orange", linewidth=2)
ax.set_ylim(-0.8, 0.8)

In [ ]:
fig, gs = plot_slices_3b.plot_slices(model, ('W','W','W'),
                                     thetas=(45, 60, 90, 120, 180),
                                     vmin=-0.6, vmax=0.6)

In [ ]:
tbp = plotting3d.ThreeBodyPlotter(model, ("W", "W", "W"))
tbp.sample_uniformly(20)
tbp.plot_uniform(val_limit=0.1)

# Prediction

In [ ]:
y_e, p_e, y_f, p_f, rmse_e, rmse_f = model.batched_predict(filename, 
                                                           keys=test_index)

In [ ]:
plotting.density_scatter(y_e, p_e)
plt.tight_layout()

In [ ]:
plotting.density_scatter(y_f, p_f)
plt.tight_layout()

## Export UF3 potential for LAMMPS

In [ ]:
# Save model in json file
model.to_json("tungsten.json")

# Call Python code "generate_uf3_lammps_pots.py" to generate the LAMMPS potential files

!python generate_uf3_lammps_pots.py \
    --author AI4Materials \
    --units metal \
    --model tungsten.json \
    --directory ./ \
    --knots_spacing_type nk

# Replace "{2: 0, 3: 0} {2: 3, 3: 3}"" in lines 6 and 9 with "0 3" in file W.uf3
import re
with open('W.uf3', 'r') as f:
    content = f.read()

# Replace all occurrences of the exact pattern
content = re.sub(r"\{2: 0, 3: 0\} \{2: 3, 3: 3\}", "0 3", content)

with open('W.uf3', 'w') as f:
    f.write(content)

## Perform molecular dynamics simulation using LAMMPS

### Installing LAMMPS with ML-UF3 in a Miniconda Environment

This guide explains how to build and run LAMMPS with the `ML-UF3` package enabled inside an existing **Miniconda environment**. This is ideal for users who already manage Python ML tools via Conda and want to extend their workflows to LAMMPS simulations.

---

#### Step 1: Activate Your Miniconda Environment

Open your terminal or VS Code integrated terminal and activate your existing environment:

- conda activate your-env-name

---
#### Step 2: Install Build Tools and Compilers

Install the necessary packages from conda-forge:
- conda install -c conda-forge cmake make
- conda install -c conda-forge compilers
- conda install -c conda-forge fftw blas

These provide:
- cmake – build system
- make – build tool
- gcc, g++, gfortran – C/C++/Fortran compilers
- FFT and BLAS libraries for performance

---
#### Step 3: Clone and Build LAMMPS with ML-UF3
- git clone https://github.com/lammps/lammps.git
- cd lammps
- mkdir build && cd build

Now configure with CMake, ensuring that ML-UF3 is enabled:
- cmake ../cmake \
  -D CMAKE_INSTALL_PREFIX=$HOME/lammps-conda \
  -D CMAKE_PREFIX_PATH=$CONDA_PREFIX \
  -D BUILD_MPI=OFF \
  -D BUILD_SHARED_LIBS=OFF \
  -D PKG_ML-UF3=ON \
  -D PKG_USER-MLIAP=ON \
  -D PKG_ML-SNAP=OFF \
  -D PKG_ML-IAP=OFF

Then build and install:
- make -j$(nproc)
- make install

Verify that LAMMPS was built with UF3 support:
- $HOME/lammps-conda/bin/lmp -h | grep ML-UF3

In [ ]:
# Create BCC tungsten 5x5x5 supercell
# Experimental Lattice constant for BCC tungsten in angstroms
a0 = 3.165

def create_bcc_supercell(a0, nx, ny, nz):
    # Basis atoms for BCC unit cell
    basis = np.array([
        [0.0, 0.0, 0.0],
        [0.5, 0.5, 0.5]
    ])

    positions = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                for atom in basis:
                    pos = a0 * (atom + [i, j, k])
                    positions.append(pos)
    return np.array(positions)

positions = create_bcc_supercell(a0, 5, 5, 5)
n_atoms = len(positions)
print(f"Generated {n_atoms} atoms.")

In [ ]:
def write_lammps_data(filename, positions, a0, nx, ny, nz):
    with open(filename, "w") as f:
        f.write("LAMMPS data file for BCC tungsten 5x5x5 supercell\n\n")
        f.write(f"{len(positions)} atoms\n")
        f.write("1 atom types\n\n")

        lx = nx * a0
        ly = ny * a0
        lz = nz * a0
        f.write(f"0.0 {lx:.6f} xlo xhi\n")
        f.write(f"0.0 {ly:.6f} ylo yhi\n")
        f.write(f"0.0 {lz:.6f} zlo zhi\n\n")

        f.write("Masses\n\n")
        f.write("1 183.84\n\n")  # Atomic mass of W

        f.write("Atoms\n\n")
        for i, pos in enumerate(positions, start=1):
            f.write(f"{i} 1 {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}\n")

write_lammps_data("tungsten.data", positions, a0, 5, 5, 5)
print("LAMMPS data file written to 'tungsten.data'")

In [ ]:
# Create LAMMPS input file for melting test            
with open("melting_test.in", "w") as f:
    f.write("""units metal
atom_style atomic
read_data tungsten.data

pair_style	uf3 3
pair_coeff	* * ./W.uf3 W
            
# Define thermo output
thermo          100
thermo_style    custom step temp pe etotal press vol

# Minimization (relax atoms and box)
reset_timestep  0
fix 1 all box/relax iso 0.0 vmax 0.001
min_style       cg
minimize        1.0e-10 1.0e-10 10000 100000
unfix           1

# Equilibration at 3000 K
velocity        all create 6000.0 12345 mom yes rot no dist gaussian
fix             equil all npt temp 3000.0 3000.0 0.1 iso 0.0 0.0 1.0
run             2000
unfix           equil

# Temperature ramp from 3000K to 5000K
timestep        0.001

# Write atomic positions every 100 steps
dump            1 all custom 100 dump.relax_heat.lammpstrj id type x y z
dump_modify     1 sort id
            
fix             2 all npt temp 3000 5000.0 0.1 iso 0.0 0.0 1.0
run             50000
unfix           2
                       
""")

# Running LAMMPS
!~/lammps-install/bin/lmp -in melting_test.in


In [ ]:
# Read the LAMMPS log file and plot the temperature vs. time
import pandas as pd

log_file = "log.lammps"

# Skip to section "Temperature ramp"
lammps_data = pd.read_csv(log_file, delim_whitespace=True, skiprows=230, on_bad_lines="skip")
lammps_data.columns = ["Step", "Temp", "Epot", "Etot", "Press", "Volume"]
lammps_data = lammps_data.dropna()
lammps_data["Temp"] = pd.to_numeric(lammps_data["Temp"], errors='coerce')
lammps_data = lammps_data.dropna()
lammps_data["Step"] = pd.to_numeric(lammps_data["Step"], errors='coerce')
lammps_data = lammps_data.dropna()
lammps_data["Time"] = lammps_data["Step"] * 0.001  # Convert to ps

print(lammps_data.head())
print(lammps_data["Epot"])

plt.figure(figsize=(10, 5))

plt.plot(lammps_data["Time"], lammps_data["Temp"], label="Epot")

plt.xlabel("Time (ps)")
plt.ylabel("Temperature (K)")
plt.title("Temperature vs. Time during Melting Test")
plt.legend()

plt.show()


### Assignment

> Which calculated parameter may help us identify the melting of the materials?
>
> Select one and create a plot of its dependence on temperature.
>
> What is the predicted melting point and how does it compare to the experimental value?
>
> What could be the reason for any discrepancy?